In [8]:
import psycopg2 as pg
import pandas as pd

# Database setup
host = "localhost"
database = "cdm"
user = "postgres"
password = %env PGPASSWORD
connection_string = "host={} dbname={} user={} password={}".format(host, database, user, password)

db = pg.connect(connection_string)

In [13]:
df = pd.read_sql('select relname as "Table", n_live_tup "Row Count" from pg_stat_user_tables', con=db)
print(df)

                    Table  Row Count
0         concept_synonym    3391372
1                specimen          0
2    concept_relationship    9254447
3                  person     116352
4       payer_plan_period     389231
5        cohort_attribute          0
6                   death       5461
7    attribute_definition          0
8           concept_class        315
9              vocabulary         27
10              care_site     239158
11              drug_cost    5552376
12      fact_relationship          0
13             cdm_source          0
14  source_to_concept_map          0
15          drug_exposure    6303448
16               dose_era          0
17               location       3088
18               provider     635456
19            measurement    3704927
20      cohort_definition          0
21               drug_era          0
22          condition_era          0
23                   note          0
24   procedure_occurrence   13926500
25          drug_strength     195736
2